# Kobling og dublettkontroll
Dette er fra grunnkurset med noe tillegg

In [1]:
import numpy as np
import pandas as pd
from io import StringIO 

## Koble filer
Vi bruker merge når vi skal koble filer. Denne fungerer som en sql join. Det betyr at alle kobler med alle med samme nøkkel. Vi kan velge om vi skal ta med de som ikke kobler.

Vi starter med å lese inn 2 filer som, når de kobles, inneholder alle varianter av koblinger. Det er disse variantene:
- 1:1
- 1:mange
- mange:1
- mange:mange
- 1:ingen
- mange:ingen
- ingen:1
- ingen:mange



In [2]:
reg1file="""
3 c 03
1 a
2 b 02
4 e 05
3 d 04
4 f 06
4 g 07
7 i 09
5 h 08
7 j 10
"""
reg1 = pd.read_csv(
        StringIO(reg1file),
        names=['id', 'mstat', 'county'],
        dtype=object,
        header=None,
        sep=' '
    )

reg2file="""
2 y 19
1 z 20
2 x 18
4 v 06
3 w 17
4 u 15
8 s 12
6 t 14
8 r 11
"""
reg2 = pd.read_csv(
        StringIO(reg2file),
        names=['id', 'cstat', 'county'],
        dtype=object,
        header=None,
        sep=' '
    )
display(reg1, reg2)

,id,mstat,county
0,3,c,03
1,1,a,NaN
2,2,b,02
3,4,e,05
4,3,d,04
5,4,f,06
6,4,g,07
7,7,i,09
8,5,h,08
9,7,j,10


,id,cstat,county
0,2,y,19
1,1,z,20
2,2,x,18
3,4,v,06
4,3,w,17
5,4,u,15
6,8,s,12
7,6,t,14
8,8,r,11


## Standard merge (inner join)
Vi starter med en standard merge. Den tilsvarer en inner join i sql. Vi ser at variabler med samme navn i de dataframes som kobles kommer med med hvert sitt nye variabelnavn

In [3]:
pd.merge(reg1, reg2, on='id')

,id,mstat,county_x,cstat,county_y
0,3,c,03,w,17
1,3,d,04,w,17
2,1,a,NaN,z,20
3,2,b,02,y,19
4,2,b,02,x,18
5,4,e,05,v,06
6,4,e,05,u,15
7,4,f,06,v,06
8,4,f,06,u,15
9,4,g,07,v,06


Resultatet ble ikke sortert på nøkkelen id. Vi kan legge til det med sort_values

In [4]:
pd.merge(reg1, reg2, on='id').sort_values('id')

,id,mstat,county_x,cstat,county_y
2,1,a,NaN,z,20
3,2,b,02,y,19
4,2,b,02,x,18
0,3,c,03,w,17
1,3,d,04,w,17
5,4,e,05,v,06
6,4,e,05,u,15
7,4,f,06,v,06
8,4,f,06,u,15
9,4,g,07,v,06


Nøklene kan ha forskjellige navn. Da angir vi navnet på nøkkelvariablene

In [5]:
pd.merge(reg1, reg2, left_on='id', right_on='id').sort_values('id')

,id,mstat,county_x,cstat,county_y
2,1,a,NaN,z,20
3,2,b,02,y,19
4,2,b,02,x,18
0,3,c,03,w,17
1,3,d,04,w,17
5,4,e,05,v,06
6,4,e,05,u,15
7,4,f,06,v,06
8,4,f,06,u,15
9,4,g,07,v,06


## Full join
Vi kan gjøre en full join. Da vil alle som ikke kobler også komme med i resultatet. Vi angir koblingstypen med how-parameteren. Det er også lurt å ta med en indikator slik at vi kan se hvor de enkelte radene kommer fra. Variabler som har samme navn i begge filene uten å være en del av nøklene får nye navn: suffiks _x og _y

In [6]:
pd.merge(reg1, reg2, on='id', how='outer', indicator=True).sort_values('id')

,id,mstat,county_x,cstat,county_y,_merge
2,1,a,NaN,z,20,both
3,2,b,02,y,19,both
4,2,b,02,x,18,both
0,3,c,03,w,17,both
1,3,d,04,w,17,both
10,4,g,07,u,15,both
9,4,g,07,v,06,both
8,4,f,06,u,15,both
6,4,e,05,u,15,both
5,4,e,05,v,06,both


Vi kan velge suffiks selv

In [44]:
pd.merge(reg1, reg2, on='id', how='outer', indicator=True, suffixes = ('_1', '_2')).sort_values('id')

,id,mstat,county_1,cstat,county_2,_merge
2,1,a,NaN,z,20,both
3,2,b,02,y,19,both
4,2,b,02,x,18,both
0,3,c,03,w,17,both
1,3,d,04,w,17,both
10,4,g,07,u,15,both
9,4,g,07,v,06,both
8,4,f,06,u,15,both
6,4,e,05,u,15,both
5,4,e,05,v,06,both


## Left join

In [7]:
pd.merge(reg1, reg2, on='id', how='left', indicator=True).sort_values('id')

,id,mstat,county_x,cstat,county_y,_merge
1,1,a,NaN,z,20,both
2,2,b,02,y,19,both
3,2,b,02,x,18,both
0,3,c,03,w,17,both
6,3,d,04,w,17,both
4,4,e,05,v,06,both
5,4,e,05,u,15,both
7,4,f,06,v,06,both
8,4,f,06,u,15,both
9,4,g,07,v,06,both


## Right join

In [8]:
pd.merge(reg1, reg2, on='id', how='right', indicator=True).sort_values('id')

,id,mstat,county_x,cstat,county_y,_merge
1,1,a,NaN,z,20,both
0,2,b,02,y,19,both
2,2,b,02,x,18,both
6,3,c,03,w,17,both
7,3,d,04,w,17,both
3,4,e,05,v,06,both
4,4,f,06,v,06,both
5,4,g,07,v,06,both
8,4,e,05,u,15,both
9,4,f,06,u,15,both


## Lagre resultatet
Vi kan lagre koblingen i en panda dataframe

In [9]:
left = pd.merge(reg1, reg2, on='id', how='left', indicator=True).sort_values('id')
left

,id,mstat,county_x,cstat,county_y,_merge
1,1,a,NaN,z,20,both
2,2,b,02,y,19,both
3,2,b,02,x,18,both
0,3,c,03,w,17,both
6,3,d,04,w,17,both
4,4,e,05,v,06,both
5,4,e,05,u,15,both
7,4,f,06,v,06,both
8,4,f,06,u,15,both
9,4,g,07,v,06,both


## Sjekke forekomster
Hvis vi bare vil se hvilke identer som er i reg2 uten å hente noe data derfra kan vi bruke et oppslag med isin og bare returnere de som er der (= True)

In [10]:
reg1.loc[reg1['id'].isin(reg2['id']) == True]

,id,mstat,county
0,3,c,03
1,1,a,NaN
2,2,b,02
3,4,e,05
4,3,d,04
5,4,f,06
6,4,g,07


Vi kan gjøre motsatt og finne de som er i reg1, men ikke i reg2 ved å snu betingelsen til False. Dette kalles en anti-join.

In [11]:
reg1.loc[reg1['id'].isin(reg2['id']) == False]

,id,mstat,county
7,7,i,09
8,5,h,08
9,7,j,10


## Sette sammen dataene radvis
Vi kan også sette sammen dataene våre etter hverandre istedenfor å koble dem. Vi kan bruke concat til det. Vi ser at variabler som ikke er i begge datasett får NaN-verdi fra det datasettet de ikke er i

In [12]:
pd.concat([reg1, reg2])

,id,mstat,county,cstat
0,3,c,03,NaN
1,1,a,NaN,NaN
2,2,b,02,NaN
3,4,e,05,NaN
4,3,d,04,NaN
5,4,f,06,NaN
6,4,g,07,NaN
7,7,i,09,NaN
8,5,h,08,NaN
9,7,j,10,NaN


## Dublettsjekk
Vi starter med å liste alle like rader. Først lager vi et datasett som har like rader, deretter bruker vi duplicated til å fjerne dem

In [30]:
reg1_4 = reg1.loc[reg1['id'] == '4']
display(reg1_4)
reg1dubl = pd.concat([reg1, reg1_4])
display(reg1dubl)
reg1dubl.loc[reg1dubl.duplicated() == False]

,id,mstat,county
3,4,e,05
5,4,f,06
6,4,g,07


,id,mstat,county
0,3,c,03
1,1,a,NaN
2,2,b,02
3,4,e,05
4,3,d,04
5,4,f,06
6,4,g,07
7,7,i,09
8,5,h,08
9,7,j,10


,id,mstat,county
0,3,c,03
1,1,a,NaN
2,2,b,02
3,4,e,05
4,3,d,04
5,4,f,06
6,4,g,07
7,7,i,09
8,5,h,08
9,7,j,10


Nå skal vi liste dubletter på id-variabelen i det opprinnelige reg1-datasettet. Vi sorterer resultatet slik at det blir lettere å tolke resultatet.
Her blir alle som ikke er første dublett listet

In [32]:
reg1[reg1.duplicated(['id'], keep='first')].sort_values(['id'])

,id,mstat,county
4,3,d,04
5,4,f,06
6,4,g,07
9,7,j,10


Her blir alle unntatt den siste listet

In [33]:
reg1[reg1.duplicated(['id'], keep='last')].sort_values(['id'])

,id,mstat,county
0,3,c,03
3,4,e,05
5,4,f,06
7,7,i,09


Nå lister vi alle dubletter på id-variabelen

In [34]:
reg1[reg1.duplicated(['id'], keep=False)].sort_values(['id'])

,id,mstat,county
0,3,c,03
4,3,d,04
3,4,e,05
5,4,f,06
6,4,g,07
7,7,i,09
9,7,j,10


Vi kan velge å beholde kun den første dubletten for hver id på datasettet

In [38]:
reg1.loc[reg1.duplicated(['id'], keep='first') == False]

,id,mstat,county
0,3,c,03
1,1,a,NaN
2,2,b,02
3,4,e,05
7,7,i,09
8,5,h,08


Vi kan velge å beholde kun den siste dubletten for hver id på datasettet

In [39]:
reg1udreg1.loc[reg1.duplicated(['id'], keep='last') == False]

,id,mstat,county
1,1,a,NaN
2,2,b,02
4,3,d,04
6,4,g,07
8,5,h,08
9,7,j,10
